# 3. Transfer learning

## Packages

In [ ]:
# ----- Tensorflow -----
import tensorflow as tf
from keras.models import Model
from keras.layers import Flatten, Dense, Input
from keras import optimizers as opt

# ----- Transfer learning -----
from keras.applications import NASNetLarge
from keras.applications import EfficientNetB0
from keras.applications import MobileNetV2

# ----- plot -----
import matplotlib.pyplot as plt
from keras.utils import plot_model

# ----- Utility functions -----
from utils import load_data, plot_hist, _convert_sample, get_features_and_labels, preprocess_data, augment_layer

# ------ Data ------
import numpy as np





## Load Data

In [ ]:
data_dir = "D:/OneDrive - Syddansk Universitet/kandidat/3_semester/Maskinlæring/ds807_anvendt_maskinlæring/eksamen/exam/patch_camelyon"
BATCH_SIZE = 32
train, test, val = load_data(data_dir, perc=10, batch_size=BATCH_SIZE)




## Transfer Learning




### Pure Feature Extraction - 1 pass


#### Hyperparameters

In [ ]:
# Hyperparams
EPOCHS = 50
LEARNING_RATE = 0.001
OPTIMIZER = opt.Nadam(learning_rate = LEARNING_RATE)




#### Build Feature Extractor with NasNet Large

#### Feature Extractor

In [ ]:
# Initilize NasNetLarge Feature Extractor
NasNet_ft_extractor = NASNetLarge(
    input_shape=(96, 96, 3),
    include_top= False,
    weights='imagenet'
)
NasNet_ft_extractor.trainable = False




#### Classifier

In [ ]:
# Build classifer
x = NasNet_ft_extractor.output
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output = Dense(2, activation='softmax')(x)
m_nasnet = Model(inputs = NasNet_ft_extractor.output, outputs = output)

# Compile model
m_nasnet.compile(
    optimizer = OPTIMIZER,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    )


    

#### Extract Features

In [ ]:
# X features
nasnet_fts_train, y_train = get_features_and_labels(train, NasNet_ft_extractor, tf.keras.applications.efficientnet.preprocess_input)
nasnet_fts_val, y_val = get_features_and_labels(val, NasNet_ft_extractor, tf.keras.applications.efficientnet.preprocess_input)




#### Train model with Nasnet Features

In [ ]:
history_nasnet = m_nasnet.fit(x = nasnet_fts_train, y = y_train, validation_data = (nasnet_fts_val, y_val), epochs = EPOCHS, batch_size = BATCH_SIZE)




### Feature extractor with MobileNetV2

#### Feature Extractor

In [ ]:
# Layers for feature extraction
mobilev2_ft_extractor = MobileNetV2(
    input_shape=(96, 96, 3),
    include_top= False,
    weights='imagenet'
)
mobilev2_ft_extractor.trainable = False




#### Classifier

In [ ]:
# Full model
x = mobilev2_ft_extractor.output
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output = Dense(2, activation='softmax')(x)
m_mobilev2 = Model(inputs = mobilev2_ft_extractor.output, outputs = output)

# Compile model
m_mobilev2.compile(
    optimizer = OPTIMIZER,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    )


    

#### Extract Features

In [ ]:
# Extract Features
mobilev2_fts_train, y_train = get_features_and_labels(train, mobilev2_ft_extractor, tf.keras.applications.mobilenet_v2.preprocess_input)
mobilev2_fts_val, y_val = get_features_and_labels(val, mobilev2_ft_extractor, tf.keras.applications.mobilenet_v2.preprocess_input)




#### Train model with mobilev2 features

In [ ]:
history_mobile_v2 = m_mobilev2.fit(x = mobilev2_fts_train, y = y_train, validation_data = (mobilev2_fts_val, y_val), epochs = EPOCHS, batch_size = BATCH_SIZE)




### Feature extractor with EfficientNet B0

#### Feature extractor

In [ ]:
# Layers for feature extraction
efficient_ft_extractor = EfficientNetB0(
    input_shape=(96, 96, 3),
    include_top= False,
    weights='imagenet'
)
efficient_ft_extractor.trainable = False




#### Classifier

In [ ]:
# Full model
x = efficient_ft_extractor.output
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output = Dense(2, activation='softmax')(x)
m_efficient = Model(inputs = efficient_ft_extractor.output, outputs = output)

# Compile model
m_efficient.compile(
    optimizer = OPTIMIZER,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    )


    

#### Extract Features

In [ ]:
# X features
efficient_fts_train, y_val = get_features_and_labels(train, efficient_ft_extractor, tf.keras.applications.efficientnet.preprocess_input)
efficient_fts_val, y_val = get_features_and_labels(val, efficient_ft_extractor, tf.keras.applications.efficientnet.preprocess_input)




#### Train model with EffcientNet B0 features

In [ ]:
history_efficient = m_efficient.fit(x = efficient_fts_train, y = y_train, validation_data = (efficient_fts_val, y_val), epochs = EPOCHS, batch_size = BATCH_SIZE)




### Plot Results

#### NasNet

In [ ]:
plot_hist(history_nasnet)




#### MobileNet V2

In [ ]:
plot_hist(history_mobile_v2)




#### EfficientNet B0

In [ ]:
plot_hist(history_efficient)




## Feature Extraction with multiple passes


### Data augmentation

In [ ]:
def do_nothing(x):
    return x
imgs, lbs = preprocess_data(train, do_nothing)

img = imgs[3]
plt.title('Original Image')
plt.imshow(img)

EPOCHS = 25




In [ ]:
aug_img = augment_layer(flip = "horizontal_and_vertical", rotation=0.5, zoom = 0.4, contrast = 0.5)(img)
plt.title("Augmented Image")
plt.imshow(aug_img)




#### MobileNet V2

In [ ]:
# Build classifer
inputs = Input(shape = (96, 96, 3))
x = augment_layer(flip = "horizontal_and_vertical", rotation=0.5, zoom = 0.4, contrast = 0.5)(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = mobilev2_ft_extractor(inputs)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output = Dense(2, activation='softmax')(x)
m_mobile_layer = Model(inputs = inputs, outputs = output)

# Compile model
m_mobile_layer.compile(
    optimizer = OPTIMIZER,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    )


    

In [ ]:
history_mobile_layer = m_mobile_layer.fit(train, validation_data= val, epochs = EPOCHS)




### EfficientNet B0

In [ ]:
# Build classifer
inputs = Input(shape = (96, 96, 3))
x = augment_layer(flip = "horizontal_and_vertical", rotation=0.5, zoom = 0.4, contrast = 0.5)(inputs)
x =  tf.keras.applications.efficientnet.preprocess_input(x)
x = efficient_ft_extractor(inputs)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output = Dense(2, activation='softmax')(x)
m_efficient_layer = Model(inputs = inputs, outputs = output)

# Compile model
m_efficient_layer.compile(
    optimizer = OPTIMIZER,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    )


    

In [ ]:
history_efficient_layer = m_efficient_layer.fit(train, validation_data = val, epochs = EPOCHS)




### Plot results

#### MobileNet

In [ ]:
plot_hist(history_mobile_layer)




#### EfficientNet

In [ ]:
plot_hist(history_efficient_layer)




## Fine Tuning

#### Log architectures to textfile

In [ ]:
with open('nasnetsummary.txt', 'w') as f:

    NasNet_ft_extractor.summary(print_fn=lambda x: f.write(x + '\n'))

with open('mobilesummary.txt', 'w') as f:

    mobilev2_ft_extractor.summary(print_fn=lambda x: f.write(x + '\n'))


    

#### Unfreeze layers

In [ ]:
# Make last 20 layers trainable
for layer in mobilev2_ft_extractor.layers[-20 : -1]:
    layer.trainable = True

In [ ]:
history_fine_tune = m_mobile_layer.fit(train, validation_data= val, epochs = 10)




#### Plot results

In [ ]:
plot_hist(history_fine_tune)




## Evaluate best models

In [ ]:
test_x, test_y = get_features_and_labels(test, NasNet_ft_extractor, tf.keras.applications.efficientnet.preprocess_input)
m_nasnet.evaluate(x = test_x, y = test_y)




In [ ]:
m_mobile_layer.evaluate(test)


